In [ ]:
!mkdir /content/Bing_data

In [ ]:
#/content/drive/MyDrive/Sync/BingImage-ACACIA_RIDGE_QLD7.zip

import zipfile

zip_file = "/content/drive/MyDrive/Bing_data/Bing_data01.zip"

try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall("/content/Bing_data/")
        print("Extracted all")
except:
    print("Invalid file")

In [ ]:
!mkdir /content/Google_data

In [ ]:
#/content/drive/MyDrive/Sync/BingImage-ACACIA_RIDGE_QLD7.zip

import zipfile

zip_file = "/content/drive/MyDrive/Google_data/Google_data01.zip"

try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall("/content/Google_data/")
        print("Extracted all")
except:
    print("Invalid file")

In [ ]:
cd /content/drive/MyDrive/image_tabular-master

In [ ]:
import fastai; fastai.__version__

'1.0.61'

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.tabular import *
from image2_tabular.core import *
from image2_tabular.dataset import *
from image2_tabular.model import *
from image2_tabular.metric import *

In [ ]:
bs = 32
imsize = 256

np.random.seed(2)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torch.nn.functional")

In [ ]:
data_path = Path("/content/sample_data/")

In [ ]:
house_price_df = pd.read_csv('/content/drive/MyDrive/QUTREProject/Brisbane_suburbs_02_sales_2019_property_GB_Prop_dist16_T_new.csv', index_col=0)
house_price_df

In [ ]:
train_df=house_price_df.sample(frac=0.8,random_state=200) #random state is a seed value
test_df=house_price_df.drop(train_df.index)
train_df.reset_index()
test_df.reset_index()

In [ ]:
train_df.reset_index(drop=True, inplace=True)
train_df

In [ ]:
test_df.reset_index(drop=True, inplace=True)
test_df

In [ ]:
# Check the length of the dataset
print(train_df.shape)
print(test_df.shape)

In [ ]:
print(train_df.columns)

In [ ]:
train_df.describe()

In [ ]:
tfms = get_transforms(do_flip=False, max_rotate=5, max_zoom=1.05)

procs = [Normalize]

size = 256

In [ ]:
# idx for validation, shared by image and tabular data
val_idx = get_valid_index(train_df)
len(val_idx)

In [ ]:
# load image data using train_df and prepare fastai LabelLists
image_path_Bing = Path("/content/Bing_data/")

image_data_Bing = (ImageList.from_df(train_df, path=image_path_Bing, cols="image_Bing")
              .split_by_idx(val_idx)
              .label_from_df(cols="price_value", label_cls=FloatList, log=True)
              .transform(tfms, size=size))

# add test data so that we can make predictions
test_image_data_Bing = ImageList.from_df(test_df, path=image_path_Bing, cols="image_Bing")
image_data_Bing.add_test(test_image_data_Bing)

In [ ]:
# show one example image
print(image_data_Bing.train[0][1])
image_data_Bing.train[0][0]

In [ ]:
# load image data using train_df and prepare fastai LabelLists
image_path_Google = Path("/content/Google_data/")

image_data_Google = (ImageList.from_df(train_df, path=image_path_Google, cols="image_Google")
              .split_by_idx(val_idx)
              .label_from_df(cols="price_value", label_cls=FloatList, log=True)
              .transform(tfms, size=size))

# add test data so that we can make predictions
test_image_data_Google = ImageList.from_df(test_df, path=image_path_Google, cols="image_Google")
image_data_Google.add_test(test_image_data_Google)

In [ ]:
# show one example image
print(image_data_Google.train[0][1])
image_data_Google.train[0][0]

In [ ]:
dep_var = 'price_value'
#cat_names = ['suburb_id_x']
cat_names = []
cont_names = ['propertyFeatures_bedrooms_x', 'propertyFeatures_bathrooms_x', 'propertyFeatures_carparks_x', 'landDetails_propertyArea_x', 'school_dist'\
              , 'hospital_dist', 'uni_dist', 'park_dist', 'mall_dist', 'CBD_dist', 'First_year_transaction', 'highway_dist', 'bus_stop_dist', \
       'ferry_terminal_dist', 'railway_station_dist', 'number_transaction', \
       'crime_rate_2019', 'state_school_percentage', \
       'class_size_achievement_pct_2019', 'Density', 'Score2019']
procs = [FillMissing, Categorify, Normalize]

In [ ]:
tab_data = (TabularList.from_df(train_df, path=data_path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(val_idx)
                           .label_from_df(cols=dep_var, label_cls=FloatList, log=True))

# add test
tab_data.add_test(TabularList.from_df(test_df, cat_names=cat_names, cont_names=cont_names,
                                      processor = tab_data.train.x.processor))

In [ ]:
tab_data.train[0]

In [ ]:
integrate_train, integrate_valid, integrate_test = get_imagetabdatasets(image_data_Google, image_data_Bing, tab_data)

In [ ]:
# package train, valid, and test datasets into a fastai databunch
bs = 64

db = DataBunch.create(integrate_train, integrate_valid, integrate_test, path=data_path, bs=bs)
db

In [ ]:
# image normalization with imagenet_stats
db.norm, db.denorm = normalize_funcs_image_tab(*imagenet_stats)
db.add_tfm(db.norm)

In [ ]:
# check the shape of one batch
x, y = next(iter(db.train_dl))
len(x)

In [ ]:
# images
x[0].shape

In [ ]:
# categorical and continuous tabular data
x[1][0].shape, x[1][1].shape

In [ ]:
# targets
y.shape

In [ ]:
# cnn model for images, use Resnet50 as an example
cnn_arch = models.resnet34

# cnn_out_sz is the output size of the cnn model that will be concatenated with tabular model output
cnn_out_sz = 128

# use fastai functions to get a cnn model
image_data_db_Google = image_data_Google.databunch()
image_data_db_Google.c = cnn_out_sz
cnn_learn_Google = cnn_learner(image_data_db_Google, cnn_arch, ps=0.2)
cnn_model_Google = cnn_learn_Google.model

In [ ]:
# cnn model for images, use Resnet50 as an example
cnn_arch = models.resnet34

# cnn_out_sz is the output size of the cnn model that will be concatenated with tabular model output
cnn_out_sz = 128

# use fastai functions to get a cnn model
image_data_db_Bing = image_data_Bing.databunch()
image_data_db_Bing.c = cnn_out_sz
cnn_learn_Bing = cnn_learner(image_data_db_Bing, cnn_arch, ps=0.2)
cnn_model_Bing = cnn_learn_Bing.model

In [ ]:
# get embedding sizes of categorical data
emb_szs = tab_data.train.get_emb_szs()

# output size of the tabular model that will be concatenated with cnn model output
tab_out_sz = 1

# use fastai functions to get a tabular model
tabular_model = TabularModel(emb_szs, len(cont_names), out_sz=tab_out_sz, layers=[], ps=0.2)
tabular_model

In [ ]:
# use gpu by default if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

In [ ]:
torch.cuda.is_available()

In [ ]:
# get an integrated model that combines the two components and concatenate their outputs
# which will pass through additional fully connected layers
cnn_out_total_sz = 2 * cnn_out_sz
integrate_model = CNNTabularModel(cnn_model_Google,
                                  cnn_model_Bing,
                                  tabular_model,
                                  layers = [cnn_out_total_sz + tab_out_sz, 1],
                                  ps=0.2,
                                  out_sz=1).to(device)

In [ ]:
# check model output dimension, should be (bs, 2)
integrate_model(*x).shape

In [ ]:
# adjust loss function weight because the dataset is extremely unbalanced
weights = [1/(1-train_df["price_value"].mean()), 1/train_df["price_value"].mean()]
#loss_func = CrossEntropyFlat(weight=torch.FloatTensor(weights).to(device))
#loss_func = CrossEntropyFlat(weight=torch.FloatTensor(weights).to(device))
class L1LossFlat(nn.SmoothL1Loss):
    def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
        return super().forward(input.view(-1), target.view(-1))

In [ ]:
# package everything in a fastai learner, add auc roc score as a metric
#learn = Learner(db, integrate_model, metrics=[accuracy, ROCAUC()], loss_func=loss_func)
learn = Learner(db, integrate_model, metrics=root_mean_squared_error)
#learn.loss_func = L1LossFlat()
learn.loss_func = MSELossFlat()

In [ ]:
# organize layer groups in order to use differential learning rates provided by fastai
# the first two layer groups are earlier layers of resnet
# the last layer group consists of the fully connected layers of cnn model, tabular model,
# and final fully connected layers for the concatenated data
learn.layer_groups = [nn.Sequential(*flatten_model(cnn_learn_Google.layer_groups[0])),
                      nn.Sequential(*flatten_model(cnn_learn_Bing.layer_groups[0])),
                      nn.Sequential(*flatten_model(cnn_learn_Google.layer_groups[1])),
                      nn.Sequential(*flatten_model(cnn_learn_Bing.layer_groups[1])),
                      nn.Sequential(*(flatten_model(cnn_learn_Google.layer_groups[2]) +
                                      flatten_model(cnn_learn_Bing.layer_groups[2]) +
                                      flatten_model(integrate_model.tabular_model) +
                                      flatten_model(integrate_model.layers)))]

In [ ]:
# find learning rate to train the last layer group first
learn.freeze()
learn.lr_find()
learn.recorder.plot(suggestion=True)

In [ ]:
min_grad_lr = learn.recorder.min_grad_lr

In [ ]:
# train
min_grad_lr = 1e-1 #learn.recorder.min_grad_lr
learn.fit_one_cycle(13, min_grad_lr)

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('TabularCNN_resnet34_freeze_GB')
learn.load('TabularCNN_resnet34_freeze_GB')

In [ ]:
# make predictions for the test set
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
print(preds)

In [ ]:
print(y)

In [ ]:
# submit predictions to kaggle
labels = [np.exp(p[0].data.item()) for p in preds]
submission = pd.DataFrame({'Id': test_df['id_combined'], 'Price': test_df['price_value']})
submission["Predicted"] = labels
submission.to_csv(data_path/'image_tab_Bing.csv', index=False)
submission.head()

In [ ]:
import pandas
import matplotlib.pyplot as plt
import statsmodels.api
import statsmodels.formula.api as sm
import scipy.stats as stats

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['Price'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Price'].corr(submission['Predicted'])))

In [ ]:
import numpy as np
Y_true = submission['Price']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
dfCopyf = test_df.copy()
dfCopyf["Predicted"] = labels


In [ ]:
# dfCopy.reset_index(drop=True, inplace=True)
dfCopyf.to_csv('/content/drive/MyDrive/QUTREProject/2019_test_results_Hedonic_CNN_Fusion_GB256_T_new_freeze.csv',index=False)
dfCopyf.to_pickle('/content/drive/MyDrive/QUTREProject/2019_test_results_Hedonic_CNN_Fusion_GB256_T_new_freeze.pkl')

In [ ]:
# unfreeze all layer groups to train the entire model using differential learning rates
learn.unfreeze()
learn.fit_one_cycle(5, slice(min_grad_lr, 1e-5))

In [ ]:
learn.recorder.plot_losses()

In [ ]:
learn.save('TabularCNN_resnet34_unfreeze_GB')
learn.load('TabularCNN_resnet34_unfreeze_GB')

In [ ]:
# make predictions for the test set
preds, y = learn.get_preds(DatasetType.Test)

In [ ]:
print(preds)

In [ ]:
print(y)

In [ ]:
# submit predictions to kaggle
labels = [np.exp(p[0].data.item()) for p in preds]
submission = pd.DataFrame({'Id': test_df['id_combined'], 'Price': test_df['price_value']})
submission["Predicted"] = labels
submission.to_csv(data_path/'image_tab.csv', index=False)
submission.head()

In [ ]:
fig = plt.figure(figsize=[10, 10])
ax = fig.add_subplot(2, 2, 1)
ax.scatter(submission['Price'], submission['Predicted'])
ax.set_xlabel('value')
ax.set_ylabel('predicted')
ax.set_title(str(submission['Price'].corr(submission['Predicted'])))

In [ ]:
import numpy as np
Y_true = submission['Price']
pred = submission['Predicted']


#RMSLE
error = np.square(np.log(pred + 1) - np.log(Y_true +1)).mean() ** 0.5
print(error)

score = 1 - error
print("SCORE For test : ",score)


In [ ]:
dfCopy = test_df.copy()
dfCopy

In [ ]:
dfCopy["Predicted"] = labels
dfCopy

In [ ]:
# dfCopy.reset_index(drop=True, inplace=True)
dfCopy.to_csv('/content/drive/MyDrive/QUTREProject/2019_test_results_Hedonic_CNN_Fusion_GB256_T_new_unfreeze.csv',index=False)
dfCopy.to_pickle('/content/drive/MyDrive/QUTREProject/2019_test_results_Hedonic_CNN_Fusion_GB256_T_new_unfreeze.pkl')